Test manager of root functionality

In [ ]:

import sys
from pathlib import Path
import seaborn as sns

In [ ]:

LCL_PATH  = str(Path().cwd().parent.parent)
ROOT_PATH = str(Path(LCL_PATH).parent.parent)
EXTERN_CMD_PATH = str(Path(ROOT_PATH))
GENERIC_PATH    = str(Path(ROOT_PATH)/"GeneticAlgorithmManager")
SPECIFIC_PATH   = str(Path(ROOT_PATH)/"extension")
print("""
root path:\t{}
local path:\t{}
generic_path:\t{}
project_specific_path:\t{}
extern_cmd_path: \t{}""".format(ROOT_PATH, LCL_PATH, GENERIC_PATH, SPECIFIC_PATH, EXTERN_CMD_PATH))

In [ ]:

# adding local_folder to the system path
sys.path.append(LCL_PATH)
sys.path.append(ROOT_PATH)
sys.path.append(GENERIC_PATH)
sys.path.append(SPECIFIC_PATH)

from sys_function import * # este in root

In [ ]:

sys_remove_modules("imshow")
sys_remove_modules("algoritm_genetic") # facem update doar daca modificam ceva in generic
sys_remove_modules("genoms") # 
sys_remove_modules("builds.mutate.mutate_insertion")
sys_remove_modules("builds.dataset_utils.dataset_ttp_man")
sys_remove_modules("builds.dataset_utils.ttp_generator")

from imshow import *
from algoritm_genetic import *
from genoms import *
from builds.mutate.mutate_insertion import *
from builds.dataset_utils.dataset_ttp_man import *
from builds.dataset_utils.ttp_generator import *

# Data aquisition

In [ ]:

ttp_generator = TTPGenerator("{}/data".format(LCL_PATH))
dataset = ttp_generator(nodes_file="NODE_COORD_SECTION.csv",
                        items_file="ITEMS_SECTION.csv")
dataset

In [ ]:

dataset_obj = DatasetTTPMan(dataset)
genoms_obj  = Genoms(genome_lenght=dataset["GENOME_LENGTH"], **{"check_freq":50, 
                                                                "elite_cmp":["tsp", "kp"], 
                                                                "tsp":(0, dataset["GENOME_LENGTH"]), "kp":(0, 2)})

In [ ]:

filename_population = "{}/population.npy".format(LCL_PATH)
with open(filename_population, 'rb') as f:
    population = np.load(f)

In [ ]:
genoms_obj.setPopulation(population.copy())
genoms_obj.setElitePos(np.array([1, 3, 4], dtype=np.int32))

In [ ]:
genoms_obj.population().shape

In [ ]:

for elite_pos in genoms_obj.getElitePos():
    print(elite_pos)
    p = population["tsp"][elite_pos]
    map_imgp = ttp_generator.putRoutesOnMap(p)
    image_show_wait(map_imgp, window_name="Parent")

# MutateInsertion

In [ ]:

mutate_obj = MutateInsertion("distance", dataset_obj, neighbors_size=20)

In [ ]:

for elite_pos in genoms_obj.getElitePos()[:1]:
    print("elite_pos", elite_pos)
    p = population["tsp"][elite_pos]
    print("population", p)
    o = mutate_obj(None, None, p.copy())
    map_imgp = ttp_generator.putRoutesOnMap(p)
    map_imgo = ttp_generator.putRoutesOnMap(o)
    mask = o!=p
    print("off_p", p[mask])
    print("off_o", o[mask])
    #print(np.unique(o, return_counts=True))
    image_show(map_imgo, window_name="Off")
    image_show_wait(map_imgp, window_name="Parent")

In [ ]:
positiveInsertion(p, 5, 243)